<a href="https://colab.research.google.com/github/Ajinkyadata/Coursera-sequence-models/blob/master/add_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, LambdaCallback, EarlyStopping

from termcolor import colored

In [0]:
chars = '0123456789+'

In [0]:
num=len(chars)

In [0]:
char_ind = dict((c,i) for i, c in enumerate(chars))
ind_char=dict((i,c) for i,c in enumerate(chars))

In [0]:
def generate():
  one=np.random.randint(0,100)
  two=np.random.randint(0,100)

  ans=str(one)+str('+')+str(two)
  label=str(one+two)

  return ans, label

In [17]:
generate()

('15+0', '15')

In [0]:
hidden=128
Tx=5

In [83]:
model =Sequential([
                   SimpleRNN(hidden, input_shape=(None,num)),
                   RepeatVector(Tx),
                   SimpleRNN(hidden, return_sequences=True),
                   TimeDistributed(Dense(num ,activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], 
              )

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_8 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [0]:
#vectorisation

def vec(eg, label):
  x=np.zeros((Tx, num))
  y=np.zeros((Tx, num))

  diff_x= Tx-len(eg)
  diff_y=Tx-len(label)

  for i,c in enumerate(eg):
    x[i+diff_x, char_ind[c]]=1
  for i in range(diff_x):
    x[i,char_ind['0']]=1

  for i,c in enumerate(label):
    y[i+diff_y, char_ind[c]]=1
  for i in range(diff_y):
    y[i,char_ind['0']]=1

    return x,y


In [53]:
egg, lbl= vec('3+43','56')
print(egg)
print(lbl)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [0]:
#devec
def devec(eg):
  result= [ind_char[np.argmax(vec)] for i, vec in enumerate(eg)]
  return ''.join(result)

In [55]:
devec(egg)

'03+43'

In [0]:
#create dataset
def create_data(num_eg=2000):
  x=np.zeros((num_eg, Tx, num))
  y=np.zeros((num_eg, Tx, num))

  for i in range(num_eg):
    q, a = generate()

    x[i],y[i] = vec(q,a)

  return x,y;

In [57]:
xx, yy= create_data()

print(xx.shape)

(2000, 5, 11)


In [61]:
print(devec(xx[9]))

43+63


In [95]:
#training

call_b= LambdaCallback(
    on_epoch_end=lambda e, l: print(e, end='\n')
    )

early= EarlyStopping(monitor='val_loss', patience=10)
model.fit(xx,yy,epochs=500, batch_size=256 ,validation_split=0.2,
          verbose=False, callbacks=[early,call_b])

#scores = model.evaluate(xx, yy,verbose=0)
print ("Train loss : %.3f" % scores[0])
print ("Train accuracy : %.3f" % scores[1])
print ("Training finished") 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Train loss : 0.079
Train accuracy : 0.705
Training finished


In [96]:
x_test, y_test= create_data(10)

pred=model.predict(x_test)
for i,hat in enumerate(pred):
  y=devec(y_test[i])
  y_hat=devec(hat)
  col='green'
  if y!=y_hat:
    col='red'
  out='input: '+devec(x_test[i])+' our prediction: '+y_hat+' actual output:'+y
  print(colored(out, col))

input: 50+41 our prediction: 00191 actual output:00091
input: 43+39 our prediction: 08182 actual output:00082
input: 94+40 our prediction: 01134 actual output:00134
input: 18+90 our prediction: 00108 actual output:00108
input: 040+8 our prediction: 08148 actual output:00048
input: 30+14 our prediction: 01144 actual output:00044
input: 06+23 our prediction: 01122 actual output:00029
input: 061+0 our prediction: 01166 actual output:00061
input: 01+17 our prediction: 01118 actual output:00018
input: 64+50 our prediction: 00104 actual output:00114
